# 中文数据处理

本python框架目的在于利用python3 对中文数据进行相关处理工作，以此能够较为稳定的处理字符编码与转换。从而方便提取有用信息。方便进行数据分析。

这里主要包含 

* 数据的整合
* 数据的清理
* 数据相关信息的提取
* 数据的匹配
* 地理位置信息的获取
* 数据分析
数据 txt文件需要放在同一目录的url文件夹下面

In [2]:
# encoding=utf-8
import pandas as pd
import numpy as np
import sys 
import os
import glob

# import scipy as sp

NameError: name 'reload' is not defined

## 一级市场数据整合

* 把多个数据文件合并成一个
* 分类提取相关类型的土地数据：工业用地
* 数据存储

In [ ]:
# load the file name txt format :
path = ".\\info\\"
# file_list =os.listdir(path)
file_list = glob.glob(path+'*.txt')
file_list
savefile=[]
for element in file_list:
    savefile.append(os.path.splitext(os.path.basename(element)))
savefile


In [ ]:
## 测试

data_fm=pd.read_table(path+'2014-10-4export.txt',sep='delimiter',header=None,encoding='utf-8')
data_fm=data_fm[0].str.split('\t').apply(pd.Series, 1)

## 设置表头，重新排版
data_fm.iloc[0]
data_fm.columns=data_fm.iloc[0]
data_fm=data_fm.reindex(data_fm.index.drop(0))
data_fm=data_fm.drop(data_fm.columns[20:], axis=1)

data_fm

In [ ]:
## 循环提取数据合并

count=0
for file_ele in file_list:
    
    print(file_ele)
    temp_data_fm = pd.read_table(file_ele,sep='delimiter',header=None,encoding='utf-8')
#     temp_data_fm = pd.read_table(file_ele,sep='delimiter',header=None,encoding='utf-8-sig')
    temp_data_fm=temp_data_fm[0].str.split('\t').apply(pd.Series, 1)
    # delete the na column 
    print(temp_data_fm.shape)
    
    temp_data_fm=temp_data_fm.drop(temp_data_fm.columns[20:], axis=1)
    
    # make column name 
    temp_data_fm.iloc[0]
    temp_data_fm.columns=temp_data_fm.iloc[0]
    temp_data_fm=temp_data_fm.reindex(temp_data_fm.index.drop(0))
    
    ######
    # merge data
    ######
    if count==0:
        data_fm=temp_data_fm
    else:
        data_fm=data_fm.append(temp_data_fm,ignore_index=True)
    
    count+=1

    


In [ ]:
##-------------------
## 删除重复数据
##-------------------

print(data_fm.shape)
data_fm = data_fm.groupby(level=0).first()
print(data_fm.shape)
print(data_fm[~data_fm['电子监管号'].apply(np.isreal)].shape)

In [ ]:
##-------------------
## 得到工业土地数据 
## 工业用地：拍卖、招标、挂牌
## 仓储用地：拍卖、挂牌
##-------------------

## 判断仓储用地的供地方式类型
data_fm[data_fm['土地用途'].str.contains('仓储用地')==True]['供地方式'].unique()
## 判断土地用途类型
data_fm['土地用途'].unique()    
    
g_data_fm=data_fm.groupby(['土地用途','供地方式'])

# g_data_fm.get_group(('工业用地','拍卖出让'))
fm_industry=pd.DataFrame(columns=data_fm.columns)
supply_way=['拍卖出让', # not so many
            '招标出让', # little
            '挂牌出让'] # large

for ele in supply_way:
    fm_industry=fm_industry.append(g_data_fm.get_group((u'工业用地',ele)),ignore_index=True)

fm_industry=fm_industry.append(g_data_fm.get_group((u'仓储用地','拍卖出让')),ignore_index=True)
fm_industry=fm_industry.append(g_data_fm.get_group((u'仓储用地','挂牌出让')),ignore_index=True)

### 存储数据

In [ ]:
## 存储一级市场数据 
data_fm.to_excel(path+'data_fm.xlsx','sheet1')
fm_industry.columns
fm_industry.to_excel(path+'fm_industry.xlsx','sheet1')

## 数据清理

* 添加索引号，方便进行定位和信息搜索
* 规整位置信息数据-GIS定位准备
* 规整土地价格、面积数据 为计算准备
* 规整容积率数据


In [ ]:
### 数据载入
# 二级市场数据
sm_industry = pd.read_excel("F:\\DATAbase\\land\\industry.xlsx")
# 一级市场数据
data_fm=pd.read_excel(path+'data_fm.xlsx')
fm_industry=pd.read_excel(path+'data_fm.xlsx')


### 添加索引号



* 重新进行reindex 
* 创建索引指标，目的在于进行匹配时，方便联系一级二级匹配的数据
* 对一、二级市场数据进行贴标签



* 一级市场：标记索引号
* 二级市场：标记索引号

二级市场： 工业 0 开头 01，02... 第一位标记土地类型

一级市场： 第一二位标记土地类型，第二位标记供地方式

土地类型：

* 工业用地-00
* 商业用地-01


供地方式：

* 拍卖出让-01
* 招标出让-02
* 挂牌出让-03




In [ ]:
## 二级市场索引

# reset the index 
print(len(sm_industry))
sm_industry=sm_industry.reset_index(drop=True) # OK 
#sm_industry.index=range(len(sm_industry)) # 可以reset
print(sm_industry.index)

# set the index column 
sm_industry['index']="00"
sm_industry['index']=sm_industry['index']+sm_industry.index.astype(str)

# print(sm_industry['index'].shape)
# sm_industry['index']

In [ ]:
## 一级市场
# reset the index 
print(len(fm_industry))
fm_industry=fm_industry.reset_index(drop=True )  
# fm_industry.reindex(index=range(len(fm_industry)))# 无法reset 
# fm_industry.index=range(len(fm_industry)) # 可以reset
print(fm_industry.index)


# set the index column 
# 土地类型 工业土地为00
fm_industry['index']="00"
# 供地方式
temp=pd.DataFrame(index=fm_industry.index,columns=[0])
temp[fm_industry['供地方式'].str.contains('拍卖出让')]="01"
temp[fm_industry['供地方式'].str.contains('招标出让')]="02"
temp[fm_industry['供地方式'].str.contains('挂牌出让')]="03"

fm_industry['index']=fm_industry['index']+temp[0]

fm_industry['index']=fm_industry['index']+fm_industry.index.astype(str)
print(fm_industry['index'].shape)

In [ ]:
## 二级市场
sm_industry[u'最后一级'].fillna("-",inplace=True)
sm_industry[u'县'].fillna("-",inplace=True)
sm_industry[u'市'].fillna("-",inplace=True)
# tt=data_sm[u'最后一级'].str.replace("-","+")

# data_sm[u'市']
sm_industry[u'最后一级']=sm_industry[u'最后一级'].str.replace("-",data_sm[u'县'].values[0])
sm_industry[u'最后一级']=sm_industry[u'最后一级'].str.replace("-",data_sm[u'市'].values[0])
sm_industry[u'最后一级']=sm_industry[u'最后一级'].str.replace("-",data_sm[u'省'].values[0])
sm_industry[u'县']=sm_industry[u'县'].str.replace("-",'')
sm_industry[u'市']=sm_industry[u'市'].str.replace("-",'')

In [ ]:
## 一级市场省、市信息抓取：
# ### 导入字典 不是很好用
# code=pd.read_excel('F:\\DATAbase\\land\\code.xls')
# ###

# tat=df['电子监管号'].str[0:6].astype(int).apply(lambda x:code[code[u'地址码']==x][u'对应地址'].values)
# # print code[code[u'地址码']==411328][u'对应地址']
# tat=list(tat)
# tat=pd.DataFrame(tat,index=df.index)
# 转化为字典进行映射

# dic_code=dict(list(code.groupby(u'地址码')))
# def mapp(x):
#     if dic_code.has_key(x): 
#         return dic_code[x][u'对应地址'].values[0] 
#     else : 
#         return ""


# tatt=df['电子监管号'].str[0:2].astype(int).map(mapp)
# df['省']=tatt
# tatt=df['电子监管号'].str[0:4].astype(int).map(mapp)
# df['市']=tatt

### 土地价格、面积、容积率的清理

In [ ]:
#######################
# 二级市场
#######################

###################
#面积 规整到平方米
###################
dd=sm_industry[sm_industry[u'面积'].str.contains(u"亩")][u'面积'].str.replace(u"亩","").astype(float)*666.67
dd=dd.append(sm_industry[sm_industry[u'面积'].str.contains(u"平方米")][u'面积'].str.replace(u"平方米","").astype(float))
dd=dd.append(sm_industry[sm_industry[u'面积'].str.contains(u"公顷")][u'面积'].str.replace(u"公顷","").astype(float)*10000)
print(dd.shape)
sm_industry[u'面积']=dd


#####################
#价格
#####################

import re
sm_industry=sm_industry[sm_industry[u'转让费'].str.contains(u"万|亿元")]
dd=sm_industry[sm_industry[u'转让费'].str.contains(u"万元")][u'转让费'].str.replace(u"万元",'').str.extract(r'(\d+)')
dd=dd.append(sm_industry[sm_industry[u'转让费'].str.contains(u"亿元")][u'转让费'].str.extract(r'(\d+)').astype(float)*10000)
sm_industry[u'转让费']=dd


####################
#容积率
####################

sm_industry[u'容积率']=sm_industry[sm_industry[u'详细信息'].str.contains(u'容积率')][u'详细信息'].str.extract(r'容积率(\d+\.\d+)')
sm_industry[u'容积率'].fillna(-999,inplace=True)
sm_industry[u'容积率'].astype(str)

In [ ]:
#######################
# 一级级市场  工业用地
#######################

######################
#面积：平方米
######################
fm_industry[u'面积(公顷)']=fm_industry[u'面积(公顷)'].astype(float)*10000


#####################
#价格：万元
#####################
fm_industry['成交价格(万元)']=fm_industry['成交价格(万元)'].astype(float)


#####
# 容积率
#####
fm_industry[u'约定容积率上限'].fillna(-999,inplace=True)
fm_industry[u'约定容积率上限'].astype(float)

#####
# 土地使用年限
#####
fm_industry.loc[fm_industry['土地使用年限'].str.contains("工业"),'土地使用年限']=50

### 存储数据

In [ ]:
## save the data 
# data_fm.to_excel(path+'data_fm.xlsx','Sheet1')
fm_industry.to_excel(path+'fm_industry.xlsx','sheet1')
sm_industry.to_excel(path+'sm_industry.xlsx','sheet1')

## 一级二级市场数据匹配

* 导入整理后的数据
* 进行切词测试
* 匹配数据
    * 行政级别定位  县级
    * 按地址匹配   初步匹配
    * 容积率，面积，精确匹配
    * 地理坐标位置转换进行匹配

In [ ]:
import pandas as pd
import numpy  as np
import jieba 

In [ ]:
## 数据载入
path = "F:\\DATAbase\\land\\HDF\\"
fm_df = pd.read_excel(path+'fm_df.xlsx')
sm_df = pd.read_excel(path+'sm_df.xlsx')

In [ ]:
################# 
# jieba 切词 
#################
# add dictionary
# keyword=[]
# datafile=open('F:\\DATAbase\\land\\tttt3.txt')
# for line in datafile.readlines():
#     keyword.append(line)
    
# datafile.close()

def filter_address(demo):
    new2_str=[u'镇',u'路',u'村',u'楼']
    #倒叙删除
    for i in range(len(demo)-1,-1,-1):
        if  any( str_ in demo[i] for str_ in new2_str) and len(demo[i])<3 :
#             print('true')
            demo[i-1]=demo[i-1]+demo[i]
            del demo[i]
    return demo

demo=list(jieba.cut((u"朝阳区幸福二村24号楼")))
xxx=filter_address(demo)
print(xxx)


In [ ]:
match_column=['mindex','location','address','lyear','area','price','plot_ratio']
match_column2=['mindex','level','mkey_1','location','address','lyear','area','price','plot_ratio']
cand_col_fm=['mindex',u'行政区',u'项目位置',u'土地使用年限',u'面积(公顷)',u'成交价格(万元)',u'约定容积率上限']
cand_col_fm2=['mindex','level','mkey_1',u'行政区',u'项目位置',u'土地使用年限',u'面积(公顷)',u'成交价格(万元)',u'约定容积率上限']
cand_col_sm=['mindex',u'县',u'地址',u'剩余年限',u'面积',u'转让费',u'容积率']
cand_col_sm2=['mindex','level','mkey_1',u'县',u'地址',u'剩余年限',u'面积',u'转让费',u'容积率']

fm_df.rename(columns=dict(zip(cand_col_fm,match_column)), inplace=True)
sm_df.rename(columns=dict(zip(cand_col_sm,match_column)), inplace=True)

## construct the match data 二级市场
match_sm_data=pd.DataFrame(columns = match_column2) # output matched pair data of second market
## construct the match data 一级市场
match_fm_data=pd.DataFrame(columns = match_column2 )

print(match_sm_data)
print(match_sm_data)

In [ ]:
print(fm_df.columns)
print(sm_df.columns)

In [ ]:
print(fm_df.columns)
print(sm_df.columns)

In [ ]:
result_fm_data=pd.DataFrame(columns=match_column2)
result_sm_data=pd.DataFrame(columns=match_column2)

for key in county_sm_dict:
    if key in county_fm_dict:
        temp_fm = county_fm_dict[key]
        temp_fm = temp_fm[match_column]
        temp_fm['level']=0
        temp_fm['mkey_1']=u""


        for j in range(len(county_sm_dict[key])):
            temp_sm=county_sm_dict[key].iloc[j]
            temp_sm=temp_sm[match_column]

            temp_sm['mkey_1']=""
            temp_sm['level']=0
            
            print(temp_sm['address'])
            temp_add=list(jieba.cut(temp_sm[u'address']))
            
            temp_add2=filter_address(temp_add) # get the candidated address

            # get the first market data-slice

            ## 标记等级 一级市场
            for ele in temp_add2: #循环过滤
                temp_fm.loc[temp_fm['address'].str.contains(ele)==True,'level']+=1

            ## get the maxed 
            max_match_level=np.max(temp_fm['level'])


            ## 标记等级 二级市场
            temp_fm['level'].fillna(0,inplace=True)
            temp_sm['level']=max_match_level


            ### 记录位置


            temp_fm.loc[temp_fm['level']>=1,'mkey_1']+=temp_fm['level'].astype(str)+"|"+temp_sm['mindex'].astype(str)+u"-"
#             print temp_fm['mkey_1']


            for ele in temp_fm.loc[temp_fm['level']==np.max(temp_fm['level']),'mindex']:
                temp_sm.loc['mkey_1']+=temp_sm['level'].astype(str)+"|"+ele.astype(str)+u"-"    



            ### 存储二级市场信息
            if temp_sm['level']>=1 :
                match_sm_data=match_sm_data.append(temp_sm,ignore_index=True)

        ### rename the dataframe    
#         temp_fm.rename(columns=dict(zip(cand_col_fm2,match_column)), inplace=True)
#         result_sm_data.rename(columns=dict(zip(cand_col_sm2,match_column)), inplace=True)

        ### save the result
        temp_fm=temp_fm[temp_fm['level']>=1]
        match_fm_data=match_fm_data.append(temp_fm,ignore_index=True)
        
        result_sm_data=[]
         

In [ ]:
print(match_sm_data.shape)
print(match_fm_data.shape)

In [ ]:
match_fm_data.to_excel(path+'match_fm.xlsx','sheet1')
match_sm_data.to_excel(path+'match_sm.xlsx','sheet1')

## GIS信息获取

本程序目的在于方便进行大批量地理位置信息转换，从地址信息转换成经纬度。具体信息可参考LBS百度云。

操作方式如下：

1. 原始数据中提取所需要的地址信息，行政区信息（县、省等）
2. 把相关地址信息和行政区信息导入到info_data中
3. 运行程序，输出两个结果，第一个为地理位置信息的dataframe,第二个则为出错信息的list.

错误与获取信息失败的处理：

* 出错信息用"-"表示，返回无效地址为：""
* "-"可以进行再次尝试,fail_list2 中存储了"-"
* ""可以进行再次尝试,fail_list 中存储了""

### 程序解释

* 函数baidu_gis是进行信息查询的预先处理，遵循LBS百度云api接口相关规定，主要进行密钥加密 （中文编码是一个急需解决的大问题）
* 多线程进行

In [ ]:
import pandas as pd
import numpy as np
import sys 
# from pandas import Series,DataFrame
import urllib
import hashlib
import requests
import math
import re
import json
import copy

In [ ]:
## 数据载入
path = "F:\\DATAbase\\land\\HDF\\"
fm_df = pd.read_excel(path+'fm_df.xlsx')
sm_df = pd.read_excel(path+'sm_df.xlsx')

In [ ]:
### 转换函数
def baidu_gis(queryStr):
    '''
    queryStr 包括了请求百api的url 地址
    以get请求为例http://api.map.baidu.com/geocoder/v2/?address=百度大厦&output=json&ak=yourak
    并且对url 转码 除了保留字符不转换，其余要进行转码加密
    
    '''
    
    GIS_result=[]
    api_link=[]
    error_count=0
    # 对queryStr进行转码，safe内的保留字符不转换
    for ele in queryStr:
        encodedStr=urllib.parse.quote(ele, safe="/:=&?#+!$,;'@()*[]")
        # 在最后直接追加上yoursk
        rawStr = encodedStr + 'DTtxldoesco94o9YZT3RuGlKarBGr7Xv'

        sn = hashlib.md5(urllib.parse.quote_plus(rawStr).encode('utf-8')).hexdigest()

        api_link.append('http://api.map.baidu.com'+ele+"&sn="+sn)
        
        
    return api_link


# In[ ]:

dict_convert={
    'preffix':'/geocoder/v2/?',
    'suffix':'&output=json&ak=9gTAEoFWvBoKHl3u3dFp5ff7',
    'title': u'title=',
    'address': u'&address=',
    'province': u'&province=',
    'city': u'&city=',
    'district': u'&disctrict='    
}
key_id_list=['title','address','province','city','district']


def convert(data):
    '''
    data is a dataframe format saving the 
    info of geo info
    
    '''
    
    row,col=data.shape
    query_info=''
    for ele in data.columns:
        query_info+=dict_convert[ele] + data[ele]

    query_info=dict_convert['preffix']+query_info+dict_convert['suffix']
    query_info.fillna("",inplace=True)
    
    return query_info
    

In [ ]:
## 地址转换
col_name_sm=[u'省',u'市',u'县',u'地址']
col_name_fm=[u'行政区',u'项目位置']
gis_name_sm=['province','city','district','address']
gis_name_fm=['district','address']

flag=input('whether sm or fm')
if flag=='sm':
    data_geo=sm_df[col_name_sm]
    data_geo.rename(columns=dict(zip(col_name_sm,gis_name_sm)),inplace=True)
else:
    data_geo=fm_df[col_name_fm]
    data_geo.rename(columns=dict(zip(col_name_fm,gis_name_fm)),inplace=True)

# data_geo.rename(columns=dict(zip(col_name,gis_name)),inplace=True)


In [ ]:
query_info=convert(data_geo)
geo_link=baidu_gis(query_info)

In [ ]:
import queue
import threading
import time
import copy, sys 

exitFlag = 0
tn=20
print('we are dealing with: %s' %flag)

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/40.0.2214.93 Safari/537.36'}
req=[]
GIS_result=[]
fail_link=[]
class myThread (threading.Thread):
    def __init__(self, threadID, name, q):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.q = q
        self.header={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.85 Safari/537.36'}
    def run(self):
        print("Starting " + self.name)
        process_data(self.name, self.q,self.header)

def process_data(threadName, q, header):
    while not exitFlag:
        queueLock.acquire()
        if not workQueue.empty():
            data = q.get()
            try: 
                req = requests.get(data)
#                 print(req.json())
                if req.json()[u'status']==211:
                    print("error occur")
                    time.sleep(1)
                    req = requests.get(data)
                    print(req.json())
                if req.json()[u'status']==211:
                    time.sleep(1)
                    req = requests.get(data)
                    print(req.json())
                try:
                    content = req.json()
                    result = content['result']
                    location = result['location']
                    x = location['lat']
                    y = location['lng']
                    temp_result=(x,y,result['confidence'])
                    GIS_result.append(copy.deepcopy(temp_result))
                except: 
                    temp_result=("","","")
                    GIS_result.append(copy.deepcopy(temp_result))
            except :
                fail_link.append(data)
                temp_result=("-","-","-")
                GIS_result.append(copy.deepcopy(temp_result))
            
            queueLock.release()
            print("%s processing:  %s" % (threadName, temp_result))
        else:
            queueLock.release()
        time.sleep(1)

threadList = ["T1", "T2", "T3","T4"]
queueLock = threading.Lock()
fm_url=[]



workQueue = queue.Queue(tn)
threads = []
threadID = 1

# 创建新线程
for tName in threadList:
    print("create thread")
    thread = myThread(threadID, tName, workQueue)
    thread.start()
    threads.append(thread)
    threadID += 1

# 填充队列
queueLock.acquire()
for word in geo_link[:tn]:
    workQueue.put(word)
queueLock.release()

# 等待队列清空
while not workQueue.empty():
    pass

# 通知线程是时候退出
exitFlag = 1

# 等待所有线程完成
for t in threads:
    t.join()
print("Exiting Main Thread")

GIS_df=pd.DataFrame(GIS_result,columns=['lat','lng','confidence'])
print(GIS_df)

print("the num of empty GIS is : ")
print(len(GIS_df[GIS_df['lat']==""]))

GIS_ind=pd.concat([data_geo,GIS_df],axis=1)



if flag=='sm':
    ## save the result in the total info data
    cand_col_sm=['mindex',u'县',u'地址',u'剩余年限',u'面积',u'转让费',u'容积率']
    sm_GIS=pd.concat([sm_df[cand_col_sm],GIS_df],axis=1)

else:
    ## save the result in the  data
    cand_col_fm=['mindex',u'行政区',u'项目位置',u'土地使用年限',u'面积(公顷)',u'成交价格(万元)',u'约定容积率上限']
    sm_GIS=pd.concat([fm_df[cand_col_fm],GIS_df],axis=1)

In [ ]:
GIS_ind=pd.concat([data_geo,GIS_df],axis=1)
GIS_ind

In [ ]:
GIS_ind.to_excel(path+'GIS_fm.xlsx','fm')
GIS_ind.to_excel(path+'GIS_fm.xlsx','fm')

## 数据分析

### 画图和数据展示

本note目的在于画出一二级市场价格差的散点图和其他相关数据

* 价格差的散点图
* 地理位置分布图


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
## 散点图

# load data
path='F:\\DATAbase\\land\\HDF\\'
fm_match=pd.read_excel(path+'match_fm.xlsx')
sm_match=pd.read_excel(path+'match_sm.xlsx')

In [ ]:
# 数据预先处理
fm_match['p_a']=fm_match['price']/fm_match['area']

sm_match['p_a']=sm_match['price']/sm_match['area']
# sm_match=sm_match[sm_match['level']>=1]


# print(fm_match.head())
# print(sm_match.head())

In [ ]:
sm_group=sm_match.groupby('location')
fm_group=fm_match.groupby('location')

In [ ]:
## 取平均数 + 数据合并
p_a_df=sm_group.p_a.mean()
p_a_df=pd.DataFrame(p_a_df)
p_a_df=p_a_df.rename(columns={'p_a':'p_a_sm'})
p_a_df=pd.concat([p_a_df,fm_group.p_a.mean()],axis=1)
p_a_df=p_a_df.rename(columns={'p_a':'p_a_fm'})

p_a_df['p_a_fm'].fillna('-',inplace=True)
p_a_df=p_a_df[p_a_df['p_a_fm'].str.contains('-')!=True]

In [ ]:
# p_a_df.plot(x=p_a_df.index,y=p_a_df[['p_a_sm','p_a_fm']],kind='scatter')
p_a_df.plot()